# Project One 

In [24]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import time
from datetime import datetime

# Load API key from .env file
load_dotenv()
API_KEY = os.getenv('API_KEY')
BASE_URL = 'https://api.polygon.io'

def fetch_stock_data(ticker, start_date, end_date, retries=5, delay=310):
    """
    Fetch historical stock data from Polygon.io API with retries and delay.
    """
    url = f"{BASE_URL}/v2/aggs/ticker/{ticker}/range/1/day/{start_date}/{end_date}?apiKey={API_KEY}"
    for attempt in range(retries):
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json().get('results', [])
            return pd.DataFrame(data)
        elif response.status_code == 429:
            print(f"Rate limit exceeded for {ticker}. Retrying in {delay} seconds...")
            time.sleep(delay)
        else:
            print(f"Error fetching data for {ticker}: {response.status_code}")
            break
    return None

def fetch_data_for_portfolio(assets, start_date, end_date):
    """
    Fetch data for all assets in the portfolio.
    """
    portfolio_data = {}
    for asset_type, ticker in assets.items():
        data = fetch_stock_data(ticker, start_date, end_date)
        if data is not None:
            portfolio_data[ticker] = data
        else:
            print(f"Failed to fetch data for {ticker}")
    return portfolio_data

def save_data_to_csv(portfolio_data):
    """
    Save the fetched data to CSV files in the 'Data' folder.
    """
    if not os.path.exists('Data'):
        os.makedirs('Data')
    
    for ticker, data in portfolio_data.items():
        # Remove 'C:' or 'I:' prefix for file naming
        filename = f"Data/{ticker.replace('C:', '').replace('I:', '')}.csv"
        data.to_csv(filename, index=False)
        print(f"Data for {ticker} saved to {filename}")

# Define the assets in the portfolio
assets = {
    "Stock1": "AAPL",
    "Stock2": "JNJ",
    "Stock3": "JPM",
    "Stock4": "MSFT",
    "ETF1": "SPY",
    "ETF2": "VTI",
    "Forex1": "C:EURUSD",
    "Forex2": "C:USDJPY",
    "MarketIndex": "I:NDX"
}

# Define the date range for fetching data
start_date = '2023-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')  # Using today's date

# Fetch data for all portfolio components
portfolio_data = fetch_data_for_portfolio(assets, start_date, end_date)

# Save the fetched data to CSV files
save_data_to_csv(portfolio_data)


Rate limit exceeded for VTI. Retrying in 310 seconds...
Data for AAPL saved to Data/AAPL.csv
Data for JNJ saved to Data/JNJ.csv
Data for JPM saved to Data/JPM.csv
Data for MSFT saved to Data/MSFT.csv
Data for SPY saved to Data/SPY.csv
Data for VTI saved to Data/VTI.csv
Data for C:EURUSD saved to Data/EURUSD.csv
Data for C:USDJPY saved to Data/USDJPY.csv
Data for I:NDX saved to Data/NDX.csv


In [56]:
# -------------------
# Step 3: Clean and Transform Data
# -------------------
import pandas as pd
import os
import shutil

def load_data(ticker):
    """
    Load data from CSV file in the 'Data' folder.
    """
    try:
        filename = ticker.replace("C:", "").replace("I:", "")
        df = pd.read_csv(f"Data/{filename}.csv")
        if df.empty:
            raise ValueError(f"No data found for {ticker}.")
        return df
    except pd.errors.EmptyDataError:
        print(f"No data in CSV file for {ticker}.")
        return pd.DataFrame()

def clean_data(df, ticker):
    """
    Clean the data: handle missing values, normalize formats, rename columns.
    """
    if df.empty:
        return df
    
    # Convert the Unix timestamp to a datetime object and remove time component
    if 't' in df.columns:
        df['Date'] = pd.to_datetime(df['t'], unit='ms').dt.date
    
    # Rename columns for better readability
    if ticker == 'I:NDX':
        df.rename(columns={
            'c': 'ClosePrice',
            'h': 'HighestPrice',
            'l': 'LowestPrice',
            'o': 'OpenPrice',
            't': 'UnixTimestamp'
        }, inplace=True)
        df = df[['Date', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp']]
    else:
        df.rename(columns={
            'v': 'Volume',
            'vw': 'VolumeWeightedAvgPrice',
            'o': 'OpenPrice',
            'c': 'ClosePrice',
            'h': 'HighestPrice',
            'l': 'LowestPrice',
            'n': 'NumberOfTransactions',
            't': 'UnixTimestamp'
        }, inplace=True)
        df = df[['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions']]
    
    # Drop rows with missing dates
    df = df.dropna(subset=['Date'])
    
    # Drop unnecessary columns if they exist
    df = df.drop(columns=['t'], errors='ignore')
    
    return df

def save_cleaned_data(ticker, df):
    """
    Save the cleaned data to CSV files in the 'Cleaned_Data' folder.
    """
    cleaned_data_folder = "Cleaned_Data"
    if not os.path.exists(cleaned_data_folder):
        os.makedirs(cleaned_data_folder)
    
    filename = f"{cleaned_data_folder}/{ticker.replace('C:', '').replace('I:', '')}.csv"
    df.to_csv(filename, index=False)
    print(f"Cleaned data for {ticker} saved to {filename}")

def clear_cleaned_data():
    """
    Clear the 'Cleaned_Data' folder if it exists.
    """
    cleaned_data_folder = "Cleaned_Data"
    if os.path.exists(cleaned_data_folder):
        shutil.rmtree(cleaned_data_folder)
        print(f"Cleared existing cleaned data in {cleaned_data_folder}")

# Example usage
if __name__ == "__main__":
    tickers = ["AAPL", "JNJ", "JPM", "MSFT", "SPY", "VTI", "C:EURUSD", "C:USDJPY", "I:NDX"]
    
    # Clear previously cleaned data
    clear_cleaned_data()
    
    cleaned_data = {}
    for ticker in tickers:
        df = load_data(ticker)
        df = clean_data(df, ticker)
        cleaned_data[ticker] = df
        save_cleaned_data(ticker, df)
        print(f"Cleaned and processed data for {ticker}:\n", df.head())


Cleared existing cleaned data in Cleaned_Data
Cleaned data for AAPL saved to Cleaned_Data/AAPL.csv
Cleaned and processed data for AAPL:
          Date       Volume  VolumeWeightedAvgPrice  OpenPrice  ClosePrice  \
0  2023-01-03  112117471.0                125.7250    130.280      125.07   
1  2023-01-04   89100633.0                126.6464    126.890      126.36   
2  2023-01-05   80716808.0                126.0883    127.130      125.02   
3  2023-01-06   87754715.0                128.1982    126.010      129.62   
4  2023-01-09   70790813.0                131.6292    130.465      130.15   

   HighestPrice  LowestPrice  UnixTimestamp  NumberOfTransactions  
0      130.9000       124.17  1672722000000               1021065  
1      128.6557       125.08  1672808400000                770042  
2      127.7700       124.76  1672894800000                665458  
3      130.2900       124.89  1672981200000                711520  
4      133.4100       129.89  1673240400000                6

# Connect to SQL Server and Load Data

In [57]:
# -------------------
# Load Data into SQL Server: Cleaned Data
# -------------------
import pyodbc
import pandas as pd

# Connection string
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=IMPRINT;'
    r'DATABASE=Project 1;'
    r'Trusted_Connection=yes;'
)

# Connect to SQL Server
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Print a success message
print("Connected to SQL Server")

# Function to load CSV data into SQL Server table
def load_csv_to_sql(file_path, table_name, columns):
    df = pd.read_csv(file_path)

    # Ensure DataFrame columns match SQL table columns
    df.columns = columns
    placeholders = ', '.join(['?' for _ in range(len(df.columns))])
    columns_str = ', '.join(columns)

    sql = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"
    
    for row in df.itertuples(index=False, name=None):
        cursor.execute(sql, row)

    conn.commit()
    print(f"Inserted data into {table_name}")

# Define the cleaned data files and their corresponding SQL table names
cleaned_data_files = {
    'Cleaned_AAPL': 'Cleaned_Data/AAPL.csv',
    'Cleaned_JNJ': 'Cleaned_Data/JNJ.csv',
    'Cleaned_JPM': 'Cleaned_Data/JPM.csv',
    'Cleaned_MSFT': 'Cleaned_Data/MSFT.csv',
    'Cleaned_SPY': 'Cleaned_Data/SPY.csv',
    'Cleaned_VTI': 'Cleaned_Data/VTI.csv',
    'Cleaned_EURUSD': 'Cleaned_Data/EURUSD.csv',
    'Cleaned_USDJPY': 'Cleaned_Data/USDJPY.csv',
    'Cleaned_NDX': 'Cleaned_Data/NDX.csv'
}

# Define the columns for each table
table_columns = {
    'Cleaned_AAPL': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_JNJ': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_JPM': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_MSFT': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_SPY': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_VTI': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_EURUSD': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_USDJPY': ['Date', 'Volume', 'VolumeWeightedAvgPrice', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp', 'NumberOfTransactions'],
    'Cleaned_NDX': ['Date', 'OpenPrice', 'ClosePrice', 'HighestPrice', 'LowestPrice', 'UnixTimestamp']
}

# Load CSV data into SQL Server cleaned tables
for table_name, file_path in cleaned_data_files.items():
    load_csv_to_sql(file_path, table_name, table_columns[table_name])

# Close the connection
cursor.close()
conn.close()

# Print a success message
print("Data loading into SQL Server completed")

Connected to SQL Server
Inserted data into Cleaned_AAPL
Inserted data into Cleaned_JNJ
Inserted data into Cleaned_JPM
Inserted data into Cleaned_MSFT
Inserted data into Cleaned_SPY
Inserted data into Cleaned_VTI
Inserted data into Cleaned_EURUSD
Inserted data into Cleaned_USDJPY
Inserted data into Cleaned_NDX
Data loading into SQL Server completed
